Use this notebook to create the labels and the path files for training.txt, validation.txt, and testing.txt

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import os
import sys
import time

In [0]:
%cd "/content/drive/My Drive/TraCAR/datasets/UFPR-ALPR"

In [0]:
start_time = time.time()
count = 0
num_images = 0
write_big_txt = False
# start_count = 0

rootdir = %pwd
images_symdir = os.path.join(rootdir, 'images')

if write_big_txt:
  f = open("training.txt", 'w')
  f.close()
  f = open("testing.txt", 'w')
  f.close()
  f = open("validation.txt", 'w')
  f.close()

prev_mode = ''
mode = ''
for subdir, dirs, files in os.walk(rootdir):
  prev_mode = mode
  mode = subdir.split('/')[-2]
  if( mode == "testing" or mode == "training" or mode == "validation"):
    if mode != prev_mode:
      print(subdir)

    # if count < start_count:
    #   count += 1
    #   continue

    f = open(mode + '.txt', 'a')
    for file in files:
      #for writing the testing, training, and validation txt files
      if file[-1] == 'g': #.png file
        orig_file = os.path.join(subdir, file)
        new_file = os.path.join(images_symdir, file)
        if not os.path.exists(new_file):
          print('cp')
          !cp "$orig_file" "$new_file"
        if write_big_txt:
          f.write(images_symdir + '/' + file + "\n")
        num_images += 1

      #for writing the label files
      if file[-1] == 't': #.txt file
        fp = open(os.path.join(subdir, file), 'r')
        line = fp.readline()
        line = fp.readline()
        bbox = line.split(" ")[1:5]

        line = fp.readline()
        class_name = line.split(" ")[1]
        if class_name == "car\n":
          id = '0'
        elif class_name == "motorcycle\n":
          id = '1'
        else:
          print('error in parsing class')
          print(class_name)
          sys.exit()
        
        fp.close()

        bbox[-1] = bbox[-1][0:-1]
        bbox = [int(bbox[i]) for i in range(4)]

        #normalize bbox
        w = 1920
        h = 1080
        bbox[0] /= w
        bbox[1] /= h
        bbox[2] /= w
        bbox[3] /= h

        #convert bbox to yolo format [xc1, yc1, w, h]
        bbox[0] = (2*bbox[0]+bbox[2])/2
        bbox[1] = (2*bbox[1]+bbox[3])/2

        bbox = [str(bbox[i]) for i in range(4)]
        
        fq = open(os.path.join(rootdir, 'labels', file), 'w')
        fq.write(id +' '+bbox[0]+' '+bbox[1]+' '+bbox[2]+' '+bbox[3])
        fq.close()

      count += 1
      if count%500 == 0:
        print('images processed out of 4501:', num_images)

    f.close()

print((time.time() - start_time)/60, ' minutes elapsed' )
    